In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import os
from datetime import datetime

In [2]:
# connect colab to google drive
drive.mount('/content/gdrive', force_remount=True)
os.chdir('/content/gdrive/My Drive/Project/data')

Mounted at /content/gdrive


## Preprocessing Netflix Prize Data

### Process netflix_movie_titles.csv and save it to processed_data_prev/netflix_movie_titles_processed.csv

In [ ]:
netflix_df = pd.read_csv("netflix_movie_titles.csv", sep=',', header = None,
                         names=['movie_id', 'year_of_release', 'title'], usecols=range(3), verbose=True,
                         index_col = "movie_id", encoding = "ISO-8859-1")
netflix_df.head()

Tokenization took: 4.62 ms
Type conversion took: 7.24 ms
Parser memory cleanup took: 0.01 ms


,year_of_release,title
movie_id,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW


In [ ]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17770 entries, 1 to 17770
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year_of_release  17763 non-null  float64
 1   title            17770 non-null  object 
dtypes: float64(1), object(1)
memory usage: 416.5+ KB


In [ ]:
netflix_df.to_csv('processed_data_prev/netflix_movie_titles_processed.csv', encoding="ISO-8859-1")

### Merge 4 combined_data txt files into 1 pandas dataframe and save it to processed_data_prev/netflix_ratings.csv

In [ ]:
start = datetime.now()
if not os.path.isfile('processed_data_prev/netflix_ratings.csv'):
    # Create a file 'data.csv' before reading it
    # Read all the files in netflix and store them in one big file('data.csv')
    # We re reading from each of the four files and appendig each rating to a global file
    data = open('processed_data_prev/netflix_ratings.csv', mode='w')
    
    row = list()
    files=['combined_data_1.txt','combined_data_2.txt', 
           'combined_data_3.txt', 'combined_data_4.txt']
    for file in files:
        print("Reading ratings from {}...".format(file))
        with open(file) as f:
            for line in f: 
                del row[:]
                line = line.strip()
                if line.endswith(':'):
                    # All below are ratings for this movie, until another movie appears.
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
        print("Done.\n")
    data.close()
print('Time taken :', datetime.now() - start)

Reading ratings from combined_data_1.txt...
Done.

Reading ratings from combined_data_2.txt...
Done.

Reading ratings from combined_data_3.txt...
Done.

Reading ratings from combined_data_4.txt...
Done.

Time taken : 0:03:10.841753


In [ ]:
print("creating the dataframe from netflix_ratings.csv file..")
netflix_ratings_df = pd.read_csv('processed_data_prev/netflix_ratings.csv', sep=',', 
                       names=['movie', 'user','rating','date'])
netflix_ratings_df.date = pd.to_datetime(netflix_ratings_df.date)
print('Done.\n')

# we are arranging the ratings according to time.
print('Sorting the dataframe by date..')
netflix_ratings_df.sort_values(by='date', inplace=True)
print('Done..')

creating the dataframe from netflix_ratings.csv file..
Done.

Sorting the dataframe by date..
Done..


In [ ]:
netflix_ratings_df.head()

,movie,user,rating,date
56431994,10341,510180,4,1999-11-11
9056171,1798,510180,5,1999-11-11
58698779,10774,510180,3,1999-11-11
48101611,8651,510180,2,1999-11-11
81893208,14660,510180,2,1999-11-11


In [ ]:
netflix_ratings_df.describe()['rating']

count    1.004805e+08
mean     3.604290e+00
std      1.085219e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [ ]:
netflix_ratings_df.shape

(100480507, 4)

In [ ]:
# just to make sure that all Nan containing rows are deleted..
print("No of Nan values in our dataframe : ", sum(netflix_ratings_df.isnull().any()))

dup_bool = netflix_ratings_df.duplicated(['movie','user','rating'])
dups = sum(dup_bool) # by considering all columns..( including timestamp)
print("There are {} duplicate rating entries in the data..".format(dups))

print("Total data ")
print("-"*50)
print("\nTotal no of ratings :",netflix_ratings_df.shape[0])
print("Total No of Users   :", len(np.unique(netflix_ratings_df.user)))
print("Total No of movies  :", len(np.unique(netflix_ratings_df.movie)))

No of Nan values in our dataframe :  0
There are 0 duplicate rating entries in the data..
Total data 
--------------------------------------------------

Total no of ratings : 100480507
Total No of Users   : 480189
Total No of movies  : 17770


## Preprocessing IMDB Data

### imdb_movie_titles.tsv

In [ ]:
imdb_df = pd.read_csv("imdb_movie_titles.tsv", sep='\t')
imdb_df["startYear"] = pd.to_numeric(imdb_df["startYear"], errors='coerce')
imdb_df.head()

<ipython-input-4-db479b35a5c2>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_df = pd.read_csv("imdb_movie_titles.tsv", sep='\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,\N,1,"Comedy,Short"


In [ ]:
# test to make sure there are overlaps between two tables' movie names
merged = imdb_df.merge(netflix_df, left_on=["primaryTitle", "startYear"], right_on=["title", "year_of_release"])
merged.shape

(10856, 11)

In [ ]:
merged.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,year_of_release,title
0,tt0003740,movie,Cabiria,Cabiria,0,1914.0,\N,148,"Adventure,Drama,History",1914.0,Cabiria
1,tt0004707,movie,Tillie's Punctured Romance,Tillie's Punctured Romance,0,1914.0,\N,82,Comedy,1914.0,Tillie's Punctured Romance
2,tt0004972,movie,The Birth of a Nation,The Birth of a Nation,0,1915.0,\N,195,"Drama,War",1915.0,The Birth of a Nation
3,tt0006864,movie,Intolerance,Intolerance: Love's Struggle Throughout the Ages,0,1916.0,\N,163,"Drama,History",1916.0,Intolerance
4,tt0010418,movie,Male and Female,Male and Female,0,1919.0,\N,116,"Adventure,Drama",1919.0,Male and Female


In [ ]:
# explode the string value in the "genre" column into multiple rows, to ensure atomocity
imdb_df = imdb_df.set_index(imdb_df.columns.drop('genres',1).tolist()).genres.str.split(',', expand=True).stack().reset_index().rename(columns={0:'genres'}).loc[:, imdb_df.columns]

In [ ]:
imdb_df.head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,\N,1,Documentary
1,tt0000001,short,Carmencita,Carmencita,0,1894.0,\N,1,Short
2,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,\N,5,Animation
3,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,\N,5,Short
4,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,\N,4,Animation
5,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,\N,4,Comedy
6,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,\N,4,Romance
7,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,\N,12,Animation
8,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,\N,12,Short
9,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,\N,1,Comedy


In [ ]:
imdb_df.to_csv('processed_data_prev/imdb_movie_titles_processed.csv')

### imdb_crew.tsv

In [ ]:
crew_df = pd.read_csv("imdb_crew.tsv", sep='\t')
crew_df.head(10)

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
5,tt0000006,nm0005690,\N
6,tt0000007,"nm0005690,nm0374658",\N
7,tt0000008,nm0005690,\N
8,tt0000009,nm0085156,nm0085156
9,tt0000010,nm0525910,\N


In [ ]:
director_df = crew_df.assign(director=crew_df['directors'].str.split(',')).explode('director')[["tconst", "director"]]
director_df = director_df[director_df["director"] != "\\N"]
director_df.head(10)

,tconst,director
0,tt0000001,nm0005690
1,tt0000002,nm0721526
2,tt0000003,nm0721526
3,tt0000004,nm0721526
4,tt0000005,nm0005690
5,tt0000006,nm0005690
6,tt0000007,nm0005690
6,tt0000007,nm0374658
7,tt0000008,nm0005690
8,tt0000009,nm0085156


In [ ]:
writer_df = crew_df.assign(writer=crew_df['writers'].str.split(',')).explode('writer')[["tconst", "writer"]]
writer_df = writer_df[writer_df["writer"] != "\\N"]
writer_df.head(10)

,tconst,writer
8,tt0000009,nm0085156
34,tt0000036,nm0410331
74,tt0000076,nm0410331
89,tt0000091,nm0617588
106,tt0000108,nm0410331
107,tt0000109,nm0410331
108,tt0000110,nm0410331
109,tt0000111,nm0410331
110,tt0000112,nm0410331
111,tt0000113,nm0410331


In [ ]:
director_df.to_csv('processed_data_prev/imdb_crew_director.csv')
writer_df.to_csv('processed_data_prev/imdb_crew_writer.csv')

### imdb_name_basics.tsv

In [ ]:
name_df = pd.read_csv("imdb_name_basics.tsv", sep='\t')
name_df.head(10)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0045537,tt0053137,tt0072308,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0056404,tt0054452,tt0049189,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0080455,tt0072562,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0083922,tt0050976,tt0050986"
5,nm0000006,Ingrid Bergman,1915,1982,"actress,soundtrack,producer","tt0036855,tt0034583,tt0038109,tt0038787"
6,nm0000007,Humphrey Bogart,1899,1957,"actor,soundtrack,producer","tt0042593,tt0043265,tt0034583,tt0037382"
7,nm0000008,Marlon Brando,1924,2004,"actor,soundtrack,director","tt0047296,tt0068646,tt0070849,tt0078788"
8,nm0000009,Richard Burton,1925,1984,"actor,soundtrack,producer","tt0087803,tt0059749,tt0061184,tt0057877"
9,nm0000010,James Cagney,1899,1986,"actor,soundtrack,director","tt0029870,tt0035575,tt0042041,tt0031867"


In [ ]:
people_df = name_df[["nconst", "primaryName", "birthYear", "deathYear"]]
people_df.head()

,nconst,primaryName,birthYear,deathYear
0,nm0000001,Fred Astaire,1899,1987
1,nm0000002,Lauren Bacall,1924,2014
2,nm0000003,Brigitte Bardot,1934,\N
3,nm0000004,John Belushi,1949,1982
4,nm0000005,Ingmar Bergman,1918,2007


In [ ]:
profession_df = name_df.assign(primaryProfession=name_df['primaryProfession'].str.split(',')).explode('primaryProfession')[["nconst", "primaryProfession"]]
profession_df.head(10)

,nconst,primaryProfession
0,nm0000001,soundtrack
0,nm0000001,actor
0,nm0000001,miscellaneous
1,nm0000002,actress
1,nm0000002,soundtrack
2,nm0000003,actress
2,nm0000003,soundtrack
2,nm0000003,music_department
3,nm0000004,actor
3,nm0000004,soundtrack


In [ ]:
known_for_df = name_df.assign(knownForTitles=name_df['knownForTitles'].str.split(',')).explode('knownForTitles')[["nconst", "knownForTitles"]]
known_for_df.head(10)

,nconst,knownForTitles
0,nm0000001,tt0045537
0,nm0000001,tt0053137
0,nm0000001,tt0072308
0,nm0000001,tt0050419
1,nm0000002,tt0117057
1,nm0000002,tt0037382
1,nm0000002,tt0038355
1,nm0000002,tt0071877
2,nm0000003,tt0056404
2,nm0000003,tt0054452


In [ ]:
people_df.to_csv('processed_data_prev/imdb_person.csv')
profession_df.to_csv('processed_data_prev/imdb_profession.csv')
known_for_df.to_csv('processed_data_prev/imdb_person_known_movies.csv')

## Final Refinements to Schema to Fit the DDL
These schemas are ok and do not need to chage: imdb_person, netflix_ratings, movie_principals, profession, director, writer, person_known_movies.

The remaining schemas need to be changed in this part.

### imdb_movie_titles & movie_genre & genre

In [3]:
imdb_movie_titles_df = pd.read_csv("processed_data_prev/imdb_movie_titles_processed.csv", index_col=0)
imdb_movie_titles_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,\N,1,Documentary
1,tt0000001,short,Carmencita,Carmencita,0,1894.0,\N,1,Short
2,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,\N,5,Animation
3,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,\N,5,Short
4,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,\N,4,Animation


In [4]:
movie_genre_df = imdb_movie_titles_df[["genres"]].drop_duplicates()
movie_genre_df.head()

,genres
0,Documentary
1,Short
2,Animation
5,Comedy
6,Romance


In [5]:
genre_df = imdb_movie_titles_df[["tconst", "genres"]]
genre_df.head()

,tconst,genres
0,tt0000001,Documentary
1,tt0000001,Short
2,tt0000002,Animation
3,tt0000002,Short
4,tt0000003,Animation


In [6]:
imdb_movie_titles_df = imdb_movie_titles_df.drop(["genres", "originalTitle"], axis=1).drop_duplicates(subset=["tconst"])
imdb_movie_titles_df.head()

,tconst,titleType,primaryTitle,isAdult,startYear,endYear,runtimeMinutes
0,tt0000001,short,Carmencita,0,1894.0,\N,1
2,tt0000002,short,Le clown et ses chiens,0,1892.0,\N,5
4,tt0000003,short,Pauvre Pierrot,0,1892.0,\N,4
7,tt0000004,short,Un bon bock,0,1892.0,\N,12
9,tt0000005,short,Blacksmith Scene,0,1893.0,\N,1


In [7]:
imdb_movie_titles_df.shape

(9658781, 7)

In [ ]:
test_join = imdb_movie_titles_df.merge(genre_df, on="tconst")
test_join.shape

(15389511, 8)

In [ ]:
imdb_movie_titles_df.to_csv('processed_data_final/imdb_movie_titles_final.csv')
genre_df.to_csv('processed_data_final/genre_final.csv')
movie_genre_df.to_csv('processed_data_final/movie_genre_final.csv')

### netflix_movie_titles

In [8]:
netflix_movie_titles_df = pd.read_csv('processed_data_prev/netflix_movie_titles_processed.csv', encoding="ISO-8859-1")
netflix_movie_titles_df.head()

,movie_id,year_of_release,title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


In [9]:
merged = netflix_movie_titles_df.merge(imdb_movie_titles_df, right_on=["primaryTitle", "startYear"], left_on=["title", "year_of_release"])
merged.head(20)

,movie_id,year_of_release,title,tconst,titleType,primaryTitle,isAdult,startYear,endYear,runtimeMinutes
0,1,2003.0,Dinosaur Planet,tt0389605,tvMiniSeries,Dinosaur Planet,0,2003.0,\N,192
1,3,1997.0,Character,tt0119448,movie,Character,0,1997.0,\N,122
2,6,1997.0,Sick,tt0120126,movie,Sick,0,1997.0,\N,90
3,7,1992.0,8 Man,tt0182668,movie,8 Man,0,1992.0,\N,83
4,12,1947.0,My Favorite Brunette,tt0039645,movie,My Favorite Brunette,0,1947.0,\N,87
5,15,1988.0,Neil Diamond: Greatest Hits Live,tt0168093,video,Neil Diamond: Greatest Hits Live,0,1988.0,\N,60
6,17,2005.0,7 Seconds,tt0417395,video,7 Seconds,0,2005.0,\N,96
7,18,1994.0,Immortal Beloved,tt0110116,movie,Immortal Beloved,0,1994.0,\N,121
8,20,1972.0,Seeta Aur Geeta,tt0078222,movie,Seeta Aur Geeta,0,1972.0,\N,162
9,21,2002.0,Strange Relations,tt11590348,tvEpisode,Strange Relations,0,2002.0,\N,30


In [10]:
df = netflix_movie_titles_df.merge(merged[["movie_id", "tconst"]], how="left", on="movie_id")
df = df[["movie_id", "year_of_release", "title", "tconst"]].drop_duplicates(subset=["movie_id"])
df.head()

,movie_id,year_of_release,title,tconst
0,1,2003.0,Dinosaur Planet,tt0389605
1,2,2004.0,Isle of Man TT 2004 Review,NaN
2,3,1997.0,Character,tt0119448
3,4,1994.0,Paula Abdul's Get Up & Dance,NaN
4,5,2004.0,The Rise and Fall of ECW,NaN


In [11]:
df.to_csv('processed_data_final/netflix_movie_titles_final.csv')

### person_profession

In [ ]:
profession_df = pd.read_csv("processed_data_final/profession_final.csv", index_col=0)
profession_df.head()

,nconst,primaryProfession
0,nm0000001,soundtrack
0,nm0000001,actor
0,nm0000001,miscellaneous
1,nm0000002,actress
1,nm0000002,soundtrack


In [ ]:
person_profession_df = profession_df[["primaryProfession"]].drop_duplicates()
person_profession_df.head()

,primaryProfession
0,soundtrack
0,actor
0,miscellaneous
1,actress
2,music_department


In [ ]:
person_profession_df.to_csv('processed_data_final/person_profession_final.csv')